In [1]:
# Import modules
import os
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

In [2]:
# Define the dataset path
train_path = "dataset/dataset/train"
test_path = "dataset/test"

In [3]:
# Define the data transforms
data_transforms = {
    "train": transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    "test": transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [4]:
# Load the dataset
train_data = ImageFolder(train_path, transform=data_transforms["train"])
test_data = ImageFolder(test_path, transform=data_transforms["test"])
test_images = sorted(os.listdir(test_path+"/test"))

In [5]:
# Set the batch size (scale according to GPU Memory)
batch_size = 64

In [6]:
# Set the num_workers and pin_memory (scale according to System Memory)
n_ws = 4
p_m = True

In [7]:
# Define the test dataloader
test_loader = DataLoader(test_data, batch_size, shuffle=False, num_workers=n_ws, pin_memory=p_m)

In [8]:
# Select the device to use for processing
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [9]:
# Load the model and move it to the device
model = torch.jit.load("models/model_scripted.pt")
model = model.to(device)

In [10]:
# Predict on test data
predictions = []

model.eval()
torch.backends.cudnn.benchmark = True
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)

        # Get the predictions
        output = model(images)
        _, predicted = torch.max(output, 1)
            
        # Append the predictions to the list
        predictions.append(predicted)

# Convert the predictions to a single list
predictions = [p.item() for sublist in predictions for p in sublist]

In [11]:
# Print the predictions and labels
for i in range(len(predictions)):
    label = test_images[i]
    prediction = train_data.classes[predictions[i]]
    print("Label: {},\tPredicted: {}".format(label, prediction))

Label: AppleCedarRust1.JPG,	Predicted: Apple___Cedar_apple_rust
Label: AppleCedarRust2.JPG,	Predicted: Apple___Cedar_apple_rust
Label: AppleCedarRust3.JPG,	Predicted: Apple___Cedar_apple_rust
Label: AppleCedarRust4.JPG,	Predicted: Apple___Cedar_apple_rust
Label: AppleScab1.JPG,	Predicted: Apple___Apple_scab
Label: AppleScab2.JPG,	Predicted: Apple___Apple_scab
Label: AppleScab3.JPG,	Predicted: Apple___Apple_scab
Label: CornCommonRust1.JPG,	Predicted: Corn_(maize)___Common_rust_
Label: CornCommonRust2.JPG,	Predicted: Corn_(maize)___Common_rust_
Label: CornCommonRust3.JPG,	Predicted: Corn_(maize)___Common_rust_
Label: PotatoEarlyBlight1.JPG,	Predicted: Potato___Early_blight
Label: PotatoEarlyBlight2.JPG,	Predicted: Potato___Early_blight
Label: PotatoEarlyBlight3.JPG,	Predicted: Potato___Early_blight
Label: PotatoEarlyBlight4.JPG,	Predicted: Potato___Early_blight
Label: PotatoEarlyBlight5.JPG,	Predicted: Potato___Early_blight
Label: PotatoHealthy1.JPG,	Predicted: Potato___healthy
Label: Po